In [ ]:
import ast
import numpy as np
import pandas as pd

import torch
# Import all libraries
import pandas as pd
import numpy as np
import re

# Huggingface transformers
import transformers
from transformers import BertModel,BertTokenizer,AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#handling html data
from bs4 import BeautifulSoup

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [ ]:
df = pd.read_csv('final_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191253 entries, 0 to 191252
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   CELEX           191253 non-null  object
 1   Text            191253 non-null  object
 2   labels          191253 non-null  object
 3   ohe_labels      191253 non-null  object
 4   gensim_summary  191253 non-null  object
 5   t5_summary      191253 non-null  object
dtypes: object(6)
memory usage: 8.8+ MB


In [ ]:
df.head()

,CELEX,Text,labels,ohe_labels,gensim_summary,t5_summary
0,21980D1231(03),21980D1231(03) Decision No 3/80 of the EEC-Ice...,Greece|agreement (EU)|accession to the Europea...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",21980D1231(03) Decision No 3/80 of the EEC-Ice...,Article 23 (1) shall be amended by the additio...
1,21986A1115(03),15.11.1986 EN Official Journal of the European...,Portugal|protocol to an agreement|accession to...,"[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",15.11.1986 EN Official Journal of the European...,15.11.1986 EN Official Journal of the European...
2,21987A0720(02),20.7.1987 EN Official Journal of the European ...,protocol to an agreement|revision of an agreement,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",20.7.1987 EN Official Journal of the European ...,20.7.1987 EN Official Journal of the European ...
3,21987D0411(04),21987D0411(04) Decision No 3/86 of the EEC-Swe...,originating product,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0020 - 0024 DECISION N° 3/86 OF THE EEC-SWEDEN...,referred to in paragraph 2: 'The importer's de...
4,21987D0411(05),21987D0411(05) Decision No 3/86 of the EEC-Swi...,originating product,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0026 - 0030 DECISION N° 3/86 OF THE EEC-AUSTRI...,referred to in paragraph 2: 'the exporter's de...


In [ ]:
X = df['Text'].values
X_gensim = df['gensim_summary'].values
X_t5 = df['t5_summary'].values

y = np.asarray([[int(val) for val in ohe[1:-1].split(',')] for ohe in df['ohe_labels'].values])

In [ ]:
from sklearn.model_selection import train_test_split
# First Split for Train and Test
x_train,x_test, x_gensim_train, x_gensim_test, x_t5_train,x_t5_test, y_train,y_test = train_test_split(X, X_gensim, X_t5, y, test_size=0.2, random_state=101,shuffle=True)
# Next split Train in to training and validation
x_tr, x_val, x_gensim_tr, x_gensim_val, x_t5_tr, x_t5_val, y_tr, y_val = train_test_split(x_train, x_gensim_train, x_t5_train, y_train, test_size=0.2, random_state=101,shuffle=True)

In [ ]:
x_t5_tr.shape ,x_t5_val.shape, x_t5_test.shape

((122401,), (30601,), (38251,))

In [ ]:
y_tr.shape, y_val.shape, y_test.shape

((122401, 91), (30601, 91), (38251, 91))

In [ ]:
class QTagDataset (Dataset):
    def __init__(self,quest,tags, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = quest
        self.labels = tags
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        #token_type_ids = inputs["token_type_ids"]
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.labels[item_idx], dtype=torch.float)
            
        } 

In [ ]:
class QTagDataModule (pl.LightningDataModule):
    
    def __init__(self,x_tr,y_tr,x_val,y_val,x_test,y_test,tokenizer,batch_size=16,max_token_len=200):
        super().__init__()
        self.tr_text = x_tr
        self.tr_label = y_tr
        self.val_text = x_val
        self.val_label = y_val
        self.test_text = x_test
        self.test_label = y_test
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self):
        self.train_dataset = QTagDataset(quest=self.tr_text, tags=self.tr_label, tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.val_dataset  = QTagDataset(quest=self.val_text,tags=self.val_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.test_dataset  = QTagDataset(quest=self.test_text,tags=self.test_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        
        
    def train_dataloader(self):
        return DataLoader (self.train_dataset,batch_size = self.batch_size,shuffle = True , num_workers=32)

    def val_dataloader(self):
        return DataLoader (self.val_dataset,batch_size= 32)

    def test_dataloader(self):
        return DataLoader (self.test_dataset,batch_size= 32)

In [ ]:
# Initialize the Bert tokenizer
BERT_MODEL_NAME = "bert-base-cased" # we will use the BERT base model(the smaller one)
Bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [ ]:
# Initialize the parameters that will be use for training
N_EPOCHS = 12
BATCH_SIZE = 8
MAX_LEN = 512
LR = 2e-05

In [ ]:
# Instantiate and set up the data_module
QTdata_module = QTagDataModule(x_t5_tr,y_tr,x_t5_val,y_val,x_t5_test,y_test,Bert_tokenizer,BATCH_SIZE, MAX_LEN)
QTdata_module.setup()

In [ ]:
class QTagClassifier(pl.LightningModule):
    # Set up the classifier
    def __init__(self, n_classes=91, steps_per_epoch=None, n_epochs=3, lr=2e-5 ):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
#         self.log = 
        
    def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        output = self.classifier(output.pooler_output)
                
        return output
    
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss

    def test_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('test_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr)
        warmup_steps = self.steps_per_epoch//3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,total_steps)

        return [optimizer], [scheduler]

In [ ]:
# Instantiate the classifier model
steps_per_epoch = len(x_tr)//BATCH_SIZE
model = QTagClassifier(n_classes=91, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss', # monitored quantity
    filepath="QTag-t5-{epoch:02d}-{val_loss:.2f}",
    save_top_k=3, #  save the top 3 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [ ]:
# Instantiate the Model Trainer
trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = [2], checkpoint_callback=checkpoint_callback, progress_bar_refresh_rate = 30)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [2]


In [ ]:
# Train the Classifier Model
trainer.fit(model, QTdata_module)

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 69 K  
2 | criterion  | BCEWithLogitsLoss | 0     
/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, val dataloader 0, d

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving latest checkpoint..


1

In [ ]:
# Evaluate the model performance on the test dataset
trainer.test(model,datamodule=QTdata_module)

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------


1

In [ ]:
# Visualize the logs using tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 248891), started 0:11:27 ago. (Use '!kill 248891' to kill it.)

# Evaluate Model Performance on Test Set 

In [ ]:
# Retreive the checkpoint path for best model
model_path = checkpoint_callback.best_model_path

'/mnt/301e182f-fb3d-4e27-866b-33279031304a/Sirojiddin/Sirojiddin/longformer/QTag-t5-epoch=11-val_loss=0.00.ckpt'

In [ ]:
len(y_test), len(x_t5_test)

(38251, 38251)

In [ ]:
# Size of Test set
print(f'Number of Texts = {len(x_t5_test)}')

Number of Texts = 38251


## setup test dataset for BERT

In [ ]:
from torch.utils.data import TensorDataset

# Tokenize all questions in x_test
input_ids = []
attention_masks = []


for quest in x_t5_test:
    encoded_quest =  Bert_tokenizer.encode_plus(
                    quest,
                    None,
                    add_special_tokens=True,
                    max_length= MAX_LEN,
                    padding = 'max_length',
                    return_token_type_ids= False,
                    return_attention_mask= True,
                    truncation=True,
                    return_tensors = 'pt'      
    )
    
    # Add the input_ids from encoded question to the list.    
    input_ids.append(encoded_quest['input_ids'])
    # Add its attention mask 
    attention_masks.append(encoded_quest['attention_mask'])
    
# Now convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_test)

# Set the batch size.  
TEST_BATCH_SIZE = 64  

# Create the DataLoader.
pred_data = TensorDataset(input_ids, attention_masks, labels)
pred_sampler = SequentialSampler(pred_data)
pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=TEST_BATCH_SIZE)

In [ ]:
pred_data[0]

(tensor([  101,   170,  5867, 10148,  4019,  1144,  1151,  4275,  1111,  6656,
          4058,  1439,   140,  2249,  3463, 10837,  1475,   119,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [ ]:
len(pred_dataloader.dataset)

38251

## Prediction on test set

In [ ]:
flat_pred_outs = 0
flat_true_labels = 0

In [ ]:
# Put model in evaluation mode
model = model.to(device) # moving model to cuda
model.eval()

# Tracking variables 
pred_outs, true_labels = [], []
#i=0
# Predict 
for batch in pred_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_attn_mask, b_labels = batch
 
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        pred_out = model(b_input_ids,b_attn_mask)
        pred_out = torch.sigmoid(pred_out)
        # Move predicted output and labels to CPU
        pred_out = pred_out.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        #i+=1
        # Store predictions and true labels
        #print(i)
        #print(outputs)
        #print(logits)
        #print(label_ids)
    pred_outs.append(pred_out)
    true_labels.append(label_ids)

In [ ]:
pred_outs[0][0]

array([0.04074564, 0.04066355, 0.05343894, 0.07235543, 0.03853646,
       0.02842202, 0.05213642, 0.0394288 , 0.04324945, 0.04775783,
       0.0402438 , 0.03536554, 0.0653128 , 0.03572519, 0.040054  ,
       0.0775635 , 0.03929054, 0.03777587, 0.03608062, 0.05188999,
       0.05284642, 0.0484773 , 0.03276329, 0.04172181, 0.05977679,
       0.05339374, 0.05942405, 0.03659158, 0.03951778, 0.06192835,
       0.0341774 , 0.06059097, 0.03623303, 0.04214007, 0.03313862,
       0.03310748, 0.04151687, 0.05154571, 0.03893898, 0.06154216,
       0.04357706, 0.03402218, 0.02968553, 0.0511012 , 0.04078838,
       0.06199748, 0.04114083, 0.04012108, 0.04608027, 0.05946463,
       0.03695019, 0.05900068, 0.08136312, 0.04919024, 0.07255334,
       0.03973217, 0.05036108, 0.04031588, 0.06239519, 0.04514301,
       0.04320864, 0.05028322, 0.03805288, 0.03911937, 0.04300772,
       0.04515687, 0.03019859, 0.03662692, 0.04654574, 0.07069708,
       0.03049545, 0.05509397, 0.04613028, 0.05757701, 0.04560

In [ ]:
# Combine the results across all batches. 
flat_pred_outs = np.concatenate(pred_outs, axis=0)

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

In [ ]:
flat_pred_outs.shape , flat_true_labels.shape

((38251, 91), (38251, 91))

## Predictions of Tags in Test set
> The predictions are in terms of logits (probabilities for each of the 16 tags). Hence we need to have a threshold value to convert these probabilities to 0 or 1.

> Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the test set.

In [ ]:
#define candidate threshold values
threshold  = np.linspace(0.01, 0.11, 10)
threshold

array([0.01      , 0.02111111, 0.03222222, 0.04333333, 0.05444444,
       0.06555556, 0.07666667, 0.08777778, 0.09888889, 0.11      ])

> Let's define a function that takes a threshold value and uses it to convert probabilities into 1 or 0.

In [ ]:
# convert probabilities into 0 or 1 based on a threshold value
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

In [ ]:
flat_pred_outs[3]

array([0.04072395, 0.04069888, 0.05460038, 0.07225024, 0.03863067,
       0.02831053, 0.05275954, 0.03973302, 0.04180772, 0.04788992,
       0.04057568, 0.0345019 , 0.06641558, 0.03567895, 0.04016204,
       0.07690175, 0.03964144, 0.03747135, 0.03662259, 0.05177235,
       0.05235192, 0.0474209 , 0.03237614, 0.04065965, 0.06015196,
       0.0522748 , 0.05981769, 0.03633968, 0.03945893, 0.06014929,
       0.0340932 , 0.06147166, 0.03749007, 0.04222951, 0.03375865,
       0.03340513, 0.04173809, 0.05108757, 0.03896749, 0.06166041,
       0.04415675, 0.03356212, 0.02972174, 0.05128891, 0.03962427,
       0.06210271, 0.04106947, 0.03997713, 0.04627237, 0.05902647,
       0.03689636, 0.06013077, 0.07906432, 0.05014914, 0.07182229,
       0.04014827, 0.05099597, 0.04050746, 0.06210772, 0.04567811,
       0.0424679 , 0.04994688, 0.03799918, 0.03845801, 0.04329298,
       0.04537695, 0.03103196, 0.03608583, 0.04618472, 0.06996269,
       0.03082286, 0.05526232, 0.04628755, 0.05788715, 0.04658

In [ ]:
flat_true_labels[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0])

In [ ]:
flat_pred_outs[10]

array([0.04076137, 0.04046511, 0.05363361, 0.07227186, 0.03840172,
       0.0283697 , 0.05203769, 0.03931345, 0.0428603 , 0.04783576,
       0.04029714, 0.03524868, 0.06543951, 0.03573531, 0.03999939,
       0.07755331, 0.03919096, 0.03779121, 0.03612575, 0.05186323,
       0.05292133, 0.04846584, 0.03278302, 0.04177319, 0.05997606,
       0.05309551, 0.05948406, 0.03657156, 0.03946758, 0.06179421,
       0.03403781, 0.06038186, 0.03636804, 0.04244922, 0.03312094,
       0.03308516, 0.04150859, 0.05160462, 0.03889619, 0.06156391,
       0.0437256 , 0.03394669, 0.02963587, 0.05115883, 0.04075086,
       0.0620892 , 0.0410568 , 0.03984202, 0.0462294 , 0.05958174,
       0.03697703, 0.05938237, 0.08120335, 0.04930763, 0.07261307,
       0.03966072, 0.050257  , 0.04020025, 0.06212728, 0.04501604,
       0.04310614, 0.05053147, 0.03810534, 0.0390325 , 0.04290275,
       0.04508967, 0.0302385 , 0.03647568, 0.04650747, 0.07057096,
       0.03061105, 0.05485988, 0.04617832, 0.05795424, 0.04582

In [ ]:
flat_true_labels[10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0])

In [ ]:
from sklearn import metrics
scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = flat_true_labels.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(flat_pred_outs,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(metrics.f1_score(y_true,y_pred))

In [ ]:
# find the optimal threshold
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.06555555555555556


## Performance Score Evaluation

In [ ]:
#predictions for optimal threshold
y_pred_labels = classify(flat_pred_outs,opt_thresh)
y_pred = np.array(y_pred_labels).ravel() # Flatten

In [ ]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.96   3398632
           1       0.05      0.15      0.08     82209

    accuracy                           0.92   3480841
   macro avg       0.51      0.54      0.52   3480841
weighted avg       0.96      0.92      0.94   3480841



In [ ]:
y = np.asarray([np.asarray(labels.split('|'), dtype='object') for labels in df['labels']], dtype='object')
y_ohe = np.asarray([[int(x) for x in labels[1:-1].split(',')] for labels in df['ohe_labels']])
labellst = []
for i in range(91):    
    llset = set(y[np.where(y_ohe[:, i]==1)][0])
    for ll in y[np.where(y_ohe[:, i]==1)]:
        llset = llset.intersection(ll)
    labellst.append(list(llset)[0])
labellst = np.asarray(labellst)
labellst

array(['Greece', 'agreement (EU)', 'accession to the European Union',
       'originating product', 'Portugal', 'protocol to an agreement',
       'Spain', 'revision of an agreement', 'EU programme',
       'third country', 'European Economic Area', 'fishing area',
       'sea fish', 'fishing rights', 'catch quota',
       'economic concentration', 'labelling', 'approximation of laws',
       'consumer information', 'public health',
       'environmental protection', 'export refund', 'action programme',
       'technical standard', 'international sanctions', 'human rights',
       'import', 'import licence', 'marketing', 'veterinary inspection',
       'agricultural product', 'exchange of information',
       'marketing standard', 'foodstuff', 'consumer protection',
       'cooperation policy', 'air transport', 'EU aid', 'EU financing',
       'EU Member State', 'provision of services',
       'disclosure of information', 'chemical product', 'indemnification',
       'health control', 

In [ ]:
len(labellst)

91

In [ ]:
def inverse_labels(arr):
    inv_labels = []
    for labels in arr:
        inv_labels.append(labellst[np.where(labels)])        
    return np.asarray(inv_labels, dtype='object')

In [ ]:
y_pred = inverse_labels(np.array(y_pred_labels))
y_act = inverse_labels(flat_true_labels)

In [ ]:
df_pred = pd.DataFrame({'Body':x_test,'Actual Tags':y_act,'Predicted Tags':y_pred})
df_pred

,Body,Actual Tags,Predicted Tags
0,29.7.2011 EN Official Journal of the European ...,"[import licence, import (EU)]","[originating product, economic concentration, ..."
1,31.7.2010 EN Official Journal of the European ...,"[State aid, Italy, control of State aid, Europ...","[originating product, economic concentration, ..."
2,17.4.2014 EN Official Journal of the European ...,"[import (EU), common organisation of markets, ...","[originating product, economic concentration, ..."
3,30.3.2015 EN Official Journal of the European ...,"[EU trade mark, trademark law, registered trad...","[originating product, sea fish, economic conce..."
4,16.11.2017 EN Official Journal of the European...,"[revision of an agreement, European Economic A...","[originating product, economic concentration, ..."
...,...,...,...
38246,27.3.2004 EN Official Journal of the European ...,[EU financing],"[originating product, economic concentration, ..."
38247,32001R2522 Commission Regulation (EC) No 2522/...,"[third country, export refund, award of contract]","[originating product, economic concentration, ..."
38248,5.2.2018 EN Official Journal of the European U...,"[European trademark, trademark law, registered...","[originating product, economic concentration, ..."
38249,"EUROPEAN COMMISSION Brussels,14.6.2018 COM(201...",[EU aid],"[originating product, economic concentration, ..."


In [ ]:
df_pred.to_csv('t5_pred.csv', index = False)

In [ ]:
x_t5_test_unique, x_t5_test_idx = np.unique(x_t5_test, return_index = True)

In [ ]:
Ids = df.iloc[df[df['t5_summary'].isin(x_t5_test_unique)]['t5_summary'].drop_duplicates().index]['CELEX']

# DataFrame for Top-5 predictions

In [ ]:
predictions = pd.DataFrame(columns = labellst, data = flat_pred_outs[x_t5_test_idx], index = Ids)
predictions.reset_index(inplace = True)
predictions.rename(columns = {'CELEX': 'id'}, inplace = True)
predictions

,id,Greece,agreement (EU),accession to the European Union,originating product,Portugal,protocol to an agreement,Spain,revision of an agreement,EU programme,...,financial aid,investment company,EU trade mark,equal treatment,European trademark,trademark law,registered trademark,interpretation of the law,action for failure to fulfil an obligation,action for annulment of an EC decision
0,21987D0411(05),0.040872,0.040514,0.053821,0.072372,0.038459,0.028424,0.052179,0.039462,0.042741,...,0.031987,0.034458,0.057367,0.027981,0.048008,0.060325,0.057900,0.041776,0.046138,0.044270
1,21987D1231(03),0.040563,0.040595,0.053587,0.072631,0.038492,0.028448,0.051794,0.039270,0.043053,...,0.031738,0.034605,0.057308,0.027951,0.048295,0.059474,0.057635,0.041494,0.046811,0.043856
2,21991D1112(07),0.040791,0.040741,0.053650,0.072343,0.038437,0.028412,0.051952,0.039419,0.042974,...,0.031821,0.034445,0.057515,0.027916,0.048153,0.059776,0.057922,0.041427,0.046698,0.044102
3,21994D1231(13),0.041030,0.040664,0.053698,0.072013,0.038641,0.028296,0.052369,0.039411,0.042994,...,0.032014,0.034531,0.057447,0.027860,0.048051,0.060411,0.058254,0.041580,0.046505,0.044197
4,21996D0111(02),0.040678,0.040683,0.053624,0.072266,0.038628,0.028451,0.052258,0.039373,0.042918,...,0.031883,0.034458,0.057388,0.027896,0.048010,0.059897,0.057972,0.041568,0.046585,0.044131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30454,E2010C0159,0.041022,0.040786,0.053425,0.072492,0.038547,0.028397,0.051855,0.039269,0.043770,...,0.031582,0.034605,0.057747,0.027996,0.048250,0.059149,0.057404,0.041621,0.046976,0.043968
30455,E2010C1209(02),0.040499,0.040612,0.054225,0.073065,0.038662,0.028357,0.051825,0.039507,0.042480,...,0.032160,0.034540,0.057267,0.027758,0.048295,0.060647,0.058059,0.041630,0.046250,0.044432
30456,E2012C0112(03),0.040566,0.040863,0.055024,0.072591,0.038762,0.028446,0.052127,0.039564,0.041725,...,0.032770,0.034455,0.057268,0.028050,0.048249,0.062102,0.058799,0.042040,0.045322,0.045072
30457,E2013C0214(02),0.040708,0.040639,0.053929,0.072438,0.038576,0.028357,0.052050,0.039397,0.042513,...,0.032136,0.034479,0.057417,0.027880,0.048084,0.060387,0.058177,0.041551,0.046288,0.044356


In [ ]:
predictions.set_index('id', inplace = True)
predictions.T

id,21987D0411(05),21987D1231(03),21991D1112(07),21994D1231(13),21996D0111(02),21996D0411(03),21997D0710(18),21999A0716(01),21999D0181,22000P0301(14),...,C2016/374/02,C2017/301/03,C2019/207/02,C2019/373/03,E2004C0305R(01),E2010C0159,E2010C1209(02),E2012C0112(03),E2013C0214(02),E2019P0003
Greece,0.040872,0.040563,0.040791,0.041030,0.040678,0.040729,0.040715,0.040651,0.040898,0.040772,...,0.040679,0.040785,0.040598,0.040567,0.040806,0.041022,0.040499,0.040566,0.040708,0.040627
agreement (EU),0.040514,0.040595,0.040741,0.040664,0.040683,0.040635,0.040679,0.040719,0.040511,0.040769,...,0.040721,0.040724,0.040861,0.040755,0.040805,0.040786,0.040612,0.040863,0.040639,0.040680
accession to the European Union,0.053821,0.053587,0.053650,0.053698,0.053624,0.053658,0.053524,0.053721,0.053682,0.054194,...,0.054072,0.054026,0.054155,0.054009,0.053938,0.053425,0.054225,0.055024,0.053929,0.053979
originating product,0.072372,0.072631,0.072343,0.072013,0.072266,0.072192,0.072331,0.072263,0.072382,0.072528,...,0.072560,0.072444,0.072891,0.072819,0.072693,0.072492,0.073065,0.072591,0.072438,0.072851
Portugal,0.038459,0.038492,0.038437,0.038641,0.038628,0.038515,0.038623,0.038608,0.038424,0.038616,...,0.038668,0.038717,0.038641,0.038577,0.038584,0.038547,0.038662,0.038762,0.038576,0.038629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trademark law,0.060325,0.059474,0.059776,0.060411,0.059897,0.060017,0.059697,0.060146,0.060002,0.061160,...,0.060400,0.060344,0.060216,0.060140,0.059824,0.059149,0.060647,0.062102,0.060387,0.060129
registered trademark,0.057900,0.057635,0.057922,0.058254,0.057972,0.057986,0.057762,0.058039,0.057737,0.058316,...,0.058131,0.057760,0.057811,0.057978,0.057655,0.057404,0.058059,0.058799,0.058177,0.057782
interpretation of the law,0.041776,0.041494,0.041427,0.041580,0.041568,0.041644,0.041495,0.041701,0.041595,0.041880,...,0.041667,0.041511,0.041615,0.041532,0.041574,0.041621,0.041630,0.042040,0.041551,0.041497
action for failure to fulfil an obligation,0.046138,0.046811,0.046698,0.046505,0.046585,0.046587,0.046673,0.046442,0.046548,0.045843,...,0.046210,0.046755,0.046609,0.046366,0.046484,0.046976,0.046250,0.045322,0.046288,0.046596


In [ ]:
pred_transform = predictions.T.copy()
pred_transform

id,21987D0411(05),21987D1231(03),21991D1112(07),21994D1231(13),21996D0111(02),21996D0411(03),21997D0710(18),21999A0716(01),21999D0181,22000P0301(14),...,C2016/374/02,C2017/301/03,C2019/207/02,C2019/373/03,E2004C0305R(01),E2010C0159,E2010C1209(02),E2012C0112(03),E2013C0214(02),E2019P0003
Greece,0.040872,0.040563,0.040791,0.041030,0.040678,0.040729,0.040715,0.040651,0.040898,0.040772,...,0.040679,0.040785,0.040598,0.040567,0.040806,0.041022,0.040499,0.040566,0.040708,0.040627
agreement (EU),0.040514,0.040595,0.040741,0.040664,0.040683,0.040635,0.040679,0.040719,0.040511,0.040769,...,0.040721,0.040724,0.040861,0.040755,0.040805,0.040786,0.040612,0.040863,0.040639,0.040680
accession to the European Union,0.053821,0.053587,0.053650,0.053698,0.053624,0.053658,0.053524,0.053721,0.053682,0.054194,...,0.054072,0.054026,0.054155,0.054009,0.053938,0.053425,0.054225,0.055024,0.053929,0.053979
originating product,0.072372,0.072631,0.072343,0.072013,0.072266,0.072192,0.072331,0.072263,0.072382,0.072528,...,0.072560,0.072444,0.072891,0.072819,0.072693,0.072492,0.073065,0.072591,0.072438,0.072851
Portugal,0.038459,0.038492,0.038437,0.038641,0.038628,0.038515,0.038623,0.038608,0.038424,0.038616,...,0.038668,0.038717,0.038641,0.038577,0.038584,0.038547,0.038662,0.038762,0.038576,0.038629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trademark law,0.060325,0.059474,0.059776,0.060411,0.059897,0.060017,0.059697,0.060146,0.060002,0.061160,...,0.060400,0.060344,0.060216,0.060140,0.059824,0.059149,0.060647,0.062102,0.060387,0.060129
registered trademark,0.057900,0.057635,0.057922,0.058254,0.057972,0.057986,0.057762,0.058039,0.057737,0.058316,...,0.058131,0.057760,0.057811,0.057978,0.057655,0.057404,0.058059,0.058799,0.058177,0.057782
interpretation of the law,0.041776,0.041494,0.041427,0.041580,0.041568,0.041644,0.041495,0.041701,0.041595,0.041880,...,0.041667,0.041511,0.041615,0.041532,0.041574,0.041621,0.041630,0.042040,0.041551,0.041497
action for failure to fulfil an obligation,0.046138,0.046811,0.046698,0.046505,0.046585,0.046587,0.046673,0.046442,0.046548,0.045843,...,0.046210,0.046755,0.046609,0.046366,0.046484,0.046976,0.046250,0.045322,0.046288,0.046596


In [ ]:
top_5 = pd.DataFrame(index = predictions.index, columns = ['Top_1', 'Top_2', 'Top_3', 'Top_4', 'Top_5', 'labels'])

for col in pred_transform.columns:
    labels = df[df['CELEX'] == col]['labels']
    top_5.loc[col] = list(predictions.T[col].nlargest(5).items()) + [tuple(labels)]
    
top_5

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


,Top_1,Top_2,Top_3,Top_4,Top_5,labels
id,,,,,,
21987D0411(05),"(merger control, 0.09653677046298981)","(State aid, 0.08071401715278625)","(economic concentration, 0.07754345238208771)","(originating product, 0.07237206399440765)","(Italy, 0.07234729081392288)","(originating product,)"
21987D1231(03),"(merger control, 0.09633835405111313)","(State aid, 0.0820336863398552)","(economic concentration, 0.07790645956993103)","(originating product, 0.0726310983300209)","(Italy, 0.07256834208965302)","(originating product,)"
21991D1112(07),"(merger control, 0.09630230814218521)","(State aid, 0.081343874335289)","(economic concentration, 0.07757087796926498)","(Italy, 0.07278932631015778)","(originating product, 0.07234343141317368)","(agreement (EU)|originating product,)"
21994D1231(13),"(merger control, 0.09585046768188477)","(State aid, 0.08084011822938919)","(economic concentration, 0.0773252546787262)","(Italy, 0.07279499620199203)","(originating product, 0.07201282680034637)","(European Economic Area,)"
21996D0111(02),"(merger control, 0.09593036770820618)","(State aid, 0.08137616515159607)","(economic concentration, 0.07763747125864029)","(Italy, 0.07246657460927963)","(originating product, 0.0722656324505806)",(European Economic Area|economic concentration...
...,...,...,...,...,...,...
E2010C0159,"(merger control, 0.0954553559422493)","(State aid, 0.08248509466648102)","(economic concentration, 0.07796387374401093)","(Italy, 0.07369402050971985)","(originating product, 0.07249200344085693)","(health control,)"
E2010C1209(02),"(merger control, 0.09761694073677063)","(State aid, 0.0802752822637558)","(economic concentration, 0.07761640846729279)","(originating product, 0.07306477427482605)","(Italy, 0.07210673391819)","(financial aid|control of State aid|State aid,)"
E2012C0112(03),"(merger control, 0.09903564304113388)","(State aid, 0.07878894358873367)","(economic concentration, 0.07746317237615585)","(originating product, 0.07259112596511841)","(Italy, 0.07165636867284775)","(control of State aid|State aid,)"


In [ ]:
top_5.to_csv('t5_top_5_pred.csv')